<a href="https://colab.research.google.com/github/naman065/SecyRecruitmentPCLUB/blob/Computer-Vision/ComputerVision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Computer Vision Task**
Run the code cell by cell!

In [21]:

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np

# *Download the DataSet from GitHub and Upload it on your Google Drive*
PS: Dont change any name while uploading otherwise the program will not work!!!

In [22]:

from google.colab import drive
drive.mount('/content/drive')

train_dir = '/content/drive/MyDrive/dataset/train'
validation_dir = '/content/drive/MyDrive/dataset/validation'
test_dir = '/content/drive/MyDrive/dataset/test'

Mounted at /content/drive


**DATA AUGMENTATION**

In [23]:

# Data augmentation and normalization for training
# I have done this to randomize the images and avoid overfitting.
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Only normalization for validation and test
val_test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

validation_generator = val_test_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


Found 288 images belonging to 2 classes.
Found 99 images belonging to 2 classes.
Found 81 images belonging to 2 classes.


**Pre-Trained Model is used and Custom Layers are added for classification into male and female**

In [24]:
#MobileNetV2 is used for this model
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

x = base_model.output
x = GlobalAveragePooling2D()(x)  #Also Done to Avoid OVERFITTING and simplify the MODEL!!!
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

9406464/9406464 [==============================] - 1s 0us/step


**Initial Layers of MobileNetV2 is freezed i.e not used and Model is Trained on the corresponding features obtained from pre trained model**

In [25]:
#Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
#I used adam optimizer from a similar code of age detection on kaggle!
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)


Epoch 1/10
9/9 [==============================] - 113s 13s/step - loss: 1.7561 - accuracy: 0.4931 - val_loss: 0.5623 - val_accuracy: 0.7071
Epoch 2/10
9/9 [==============================] - 6s 637ms/step - loss: 0.6676 - accuracy: 0.6667 - val_loss: 0.6970 - val_accuracy: 0.5253
Epoch 3/10
9/9 [==============================] - 5s 500ms/step - loss: 0.5840 - accuracy: 0.7014 - val_loss: 0.4977 - val_accuracy: 0.8182
Epoch 4/10
9/9 [==============================] - 5s 583ms/step - loss: 0.4861 - accuracy: 0.7674 - val_loss: 0.6868 - val_accuracy: 0.5455
Epoch 5/10
9/9 [==============================] - 5s 504ms/step - loss: 0.4533 - accuracy: 0.7743 - val_loss: 0.4610 - val_accuracy: 0.7677
Epoch 6/10
9/9 [==============================] - 6s 693ms/step - loss: 0.4154 - accuracy: 0.8056 - val_loss: 0.6182 - val_accuracy: 0.6667
Epoch 7/10
9/9 [==============================] - 4s 499ms/step - loss: 0.3701 - accuracy: 0.8576 - val_loss: 0.4795 - val_accuracy: 0.7475
Epoch 8/10
9/9 [====

***THIS WAS WRITTEN TO IMPROVE UPON ACCURACY DONT RUN AS TAKES TIME P:***

In [ ]:
"""
for layer in base_model.layers[-20:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)"""


**DataSet also includes a test file to test the accuracy of the model**

In [26]:

test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy:.2f}')
print(f'Test Loss: {test_loss:.2f}')

3/3 [==============================] - 25s 12s/step - loss: 0.4141 - accuracy: 0.8519
Test Accuracy: 0.85
Test Loss: 0.41


# **Upload the image from your computer provided on GitHub**

In [30]:

from google.colab import files
from tensorflow.keras.preprocessing import image
import numpy as np

uploaded_image = files.upload()
img_path = list(uploaded_image.keys())[0]

# Image is pre processed here according to model
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0

# Prediction (male or female)
prediction = model.predict(img_array)
print("Prediction:", "Male" if prediction[0][0] > 0.5 else "Female")


Saving 00059_Mask.jpg to 00059_Mask.jpg
1/1 [==============================] - 0s 21ms/step
Prediction: Female
